### DNABERT 코드 (점수 : 0.531)

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import random
import numpy as np
import pandas as pd
from typing import List

import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from torch import amp


# =========================================
# 0) 설정 (경로 & 하이퍼파라미터)
# =========================================
SEED = 42

# ✅ DNABERT-6 (k=6) 백본
MODEL_ID = "zhihan1996/DNA_bert_6"

# 🔧 네 PC 경로에 맞게 수정
DATA_DIR = r"C:\Users\yutt4\OneDrive\바탕 화면\인공지능\open"
GRCH38_CSV = os.path.join(DATA_DIR, "grch38_windows_seq.csv")

TEST_PATH       = os.path.join(DATA_DIR, "test.csv")
SAMPLE_SUB_PATH = os.path.join(DATA_DIR, "sample_submission.csv")
OUT_PATH        = os.path.join(DATA_DIR, "submission_dnabert6_multipos_infonce_grch38_모델 변경.csv")

# 학습용 하이퍼파라미터
MAX_GRCH_SEQ   = 300_000   # GRCh38에서 최대 사용할 시퀀스 개수
CHUNK_SIZE     = 200_000   # 스트리밍 로딩 chunksize
MAX_LENGTH     = 512       # DNABERT 토큰(max_length)
KMER           = 6         # DNABERT-6 → k=6
LAST_N_LAYERS  = 4         # 마지막 4개 레이어 가중합
OUTPUT_DIM     = 512       # 제출 임베딩 차원 (<= 2048 규칙)

TRAIN_EPOCHS   = 4         # 학습 epoch 수 (시간 보고 조절)
NUM_TUPLES     = 60_000    # epoch당 (anchor, pos1, pos2) 튜플 개수
BATCH_SIZE_TR  = 32        # 학습 배치
BATCH_SIZE_INFER = 32      # 추론 배치

LR_HEAD        = 3e-4
WEIGHT_DECAY   = 1e-4
TEMPERATURE    = 0.07
USE_FP16       = True


# =========================================
# 1) 유틸 함수들
# =========================================
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def l2_normalize(x: torch.Tensor, eps: float = 1e-12) -> torch.Tensor:
    return x / x.norm(p=2, dim=-1, keepdim=True).clamp(min=eps)


_rc_map = str.maketrans("ACGT", "TGCA")
def reverse_complement(seq: str) -> str:
    return seq.translate(_rc_map)[::-1]


def clean_seq(seq: str) -> str:
    # N 제거 + 대문자
    s = str(seq).upper().replace("N", "")
    return s


def crop_seq(seq: str, max_len_nt: int) -> str:
    """
    뉴클레오타이드 기준으로 crop. (k-mer 전에 자르기)
    max_len_nt는 대략 k * MAX_LENGTH 보다 조금 크게 잡을 수도 있지만
    여기서는 그냥 512~1000 정도로 사용해도 무방.
    """
    if len(seq) <= max_len_nt:
        return seq
    start = random.randint(0, len(seq) - max_len_nt)
    return seq[start:start + max_len_nt]


def apply_snv(seq: str, k: int) -> str:
    """
    seq에서 k개의 위치를 다른 염기로 치환 (SNV augmentation)
    """
    bases = ["A", "C", "G", "T"]
    if len(seq) == 0:
        return seq
    s = list(seq)
    idxs = random.sample(range(len(s)), min(k, len(s)))
    for idx in idxs:
        orig = s[idx]
        cand = [b for b in bases if b != orig]
        if cand:
            s[idx] = random.choice(cand)
    return "".join(s)


def to_kmer(seq: str, k: int = 6) -> str:
    """
    DNABERT-6는 공백으로 구분된 k-mer 시퀀스를 토큰화함.
    예: "ACGTG..." → "ACGTGA CGTGAC ..."
    """
    seq = clean_seq(seq)
    if len(seq) < k:
        return seq
    kmers = [seq[i:i+k] for i in range(len(seq) - k + 1)]
    return " ".join(kmers)


# =========================================
# 2) GRCh38 스트리밍 로드
# =========================================
def load_grch38_sequences_stream(path: str,
                                 max_samples: int,
                                 chunksize: int = 200_000) -> List[str]:
    """
    GRCh38 windows를 chunk 단위로 읽으면서,
    최대 max_samples 개의 clean된 seq만 수집.
    """
    sequences = []
    total_rows = 0

    print(f"\n[GRCh38 STREAM LOAD] {path}")
    for chunk in pd.read_csv(path, usecols=["seq"], chunksize=chunksize):
        total_rows += len(chunk)
        seqs = [clean_seq(s) for s in chunk["seq"].astype(str).tolist()]
        seqs = [s for s in seqs if len(s) > 0]

        needed = max_samples - len(sequences)
        if needed <= 0:
            break

        if len(seqs) > needed:
            seqs = seqs[:needed]

        sequences.extend(seqs)

        del chunk
        if len(sequences) >= max_samples:
            break

    print(f"Total rows read (approx): {total_rows}")
    print("GRCh38 sequences used:", len(sequences))
    return sequences


# =========================================
# 3) multi-positive 튜플 생성
# =========================================
def generate_multi_pos_tuples(seqs: List[str], num_tuples: int) -> List[tuple]:
    """
    각 튜플: (anchor, pos1, pos2)
      - anchor: GRCh38에서 랜덤 seq 하나 사용 (crop 추가)
      - pos1  : anchor에 1~2개의 SNV
      - pos2  : reverse complement(anchor SNV 버전)
    """
    print(f"Generating {num_tuples} multi-positive tuples (anchor, pos1, pos2)")
    tuples = []
    max_anchor_nt = 512  # anchor 뉴클레오타이드 길이 상한 (적당히)

    while len(tuples) < num_tuples:
        anchor_raw = random.choice(seqs)
        anchor = crop_seq(clean_seq(anchor_raw), max_anchor_nt)
        if len(anchor) < KMER:
            continue

        # pos1: 작은 SNV (1~2개)
        pos1 = apply_snv(anchor, random.randint(1, 2))

        # pos2: RC + SNV (1개 정도)
        anchor_snv_for_rc = apply_snv(anchor, 1)
        pos2 = reverse_complement(anchor_snv_for_rc)

        tuples.append((anchor, pos1, pos2))

    return tuples


# =========================================
# 4) DNABERT 임베딩 헤드
# =========================================
class DnaBertMultiLayerHead(nn.Module):
    """
    - DNABERT hidden_states의 마지막 N개 레이어를 learnable weight로 가중합
    - attention_mask 기반 mean pooling
    - OUTPUT_DIM으로 투영 후 L2 normalize
    """
    def __init__(self, hidden_size: int, last_n: int, out_dim: int):
        super().__init__()
        self.last_n = last_n
        self.layer_weights = nn.Parameter(torch.zeros(last_n))

        self.proj = nn.Sequential(
            nn.Linear(hidden_size, hidden_size * 2),
            nn.LayerNorm(hidden_size * 2),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size * 2, out_dim)
        )

    def forward(self, hidden_states, attention_mask: torch.Tensor):
        # hidden_states: list of (B, L, H), len = num_layers+1
        hs = hidden_states[-self.last_n:]
        stack = torch.stack(hs, dim=0)  # (N, B, L, H)

        w = F.softmax(self.layer_weights, dim=0).view(-1, 1, 1, 1)
        feat = (stack * w).sum(dim=0)   # (B, L, H)

        mask = attention_mask.unsqueeze(-1).float()  # (B, L, 1)
        feat = feat * mask

        summed = feat.sum(dim=1)                     # (B, H)
        denom = mask.sum(dim=1).clamp(min=1e-9)      # (B, 1)
        mean_emb = summed / denom                    # (B, H)

        return l2_normalize(self.proj(mean_emb))     # (B, D)


# =========================================
# 5) multi-positive InfoNCE loss
# =========================================
def multipos_infonce_loss(e_anchor: torch.Tensor,
                          e_pos1: torch.Tensor,
                          e_pos2: torch.Tensor,
                          temperature: float = 0.07) -> torch.Tensor:
    """
    - anchors: (B, D)
    - pos1, pos2: (B, D)
    positives = {pos1[i], pos2[i]} for anchor[i]
    negatives = 모든 pos1/pos2 중 나머지들 (in-batch negative)
    """
    B = e_anchor.size(0)
    device = e_anchor.device

    # (B, 2B) similarity
    e_pos_all = torch.cat([e_pos1, e_pos2], dim=0)   # (2B, D)
    logits = torch.matmul(e_anchor, e_pos_all.T) / temperature  # (B, 2B)

    # 안정적인 softmax를 위해 max 빼기
    logits_max, _ = logits.max(dim=1, keepdim=True)
    logits = logits - logits_max

    exp_logits = torch.exp(logits)                  # (B, 2B)

    # positive mask: (B, 2B)
    pos_mask = torch.zeros_like(exp_logits)
    idx = torch.arange(B, device=device)
    pos_mask[idx, idx] = 1          # anchor i ↔ pos1[i]
    pos_mask[idx, idx + B] = 1      # anchor i ↔ pos2[i]

    pos_exp = (exp_logits * pos_mask).sum(dim=1)       # (B,)
    all_exp = exp_logits.sum(dim=1)                    # (B,)

    loss = -torch.log(pos_exp / all_exp + 1e-12).mean()
    return loss


# =========================================
# 6) MAIN
# =========================================
def main():

    set_seed(SEED)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Device:", device)

    # -----------------------------
    # GRCh38 로드
    # -----------------------------
    if not os.path.exists(GRCH38_CSV):
        raise FileNotFoundError(f"GRCh38 CSV를 찾지 못했습니다: {GRCH38_CSV}")

    grch_seqs = load_grch38_sequences_stream(GRCH38_CSV, max_samples=MAX_GRCH_SEQ)

    # -----------------------------
    # test 로드
    # -----------------------------
    test_df = pd.read_csv(TEST_PATH)
    test_seqs = test_df["seq"].astype(str).tolist()
    print("Test seqs:", len(test_seqs))

    # -----------------------------
    # DNABERT-6 로드
    # -----------------------------
    print("\n[Load DNABERT-6 backbone]")
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_ID,
        trust_remote_code=True,
        do_lower_case=False
    )

    backbone = AutoModel.from_pretrained(
        MODEL_ID,
        trust_remote_code=True,
        output_hidden_states=True
    ).to(device)

    backbone.eval()
    for p in backbone.parameters():
        p.requires_grad = False  # 백본 완전 freeze

    # -----------------------------
    # Head 정의
    # -----------------------------
    head = DnaBertMultiLayerHead(
        hidden_size=backbone.config.hidden_size,
        last_n=LAST_N_LAYERS,
        out_dim=OUTPUT_DIM
    ).to(device)

    optimizer = torch.optim.AdamW(
        head.parameters(),
        lr=LR_HEAD,
        weight_decay=WEIGHT_DECAY
    )

    scaler = amp.GradScaler(enabled=USE_FP16)

    # -----------------------------
    # TRAIN
    # -----------------------------
    print("\n===== multi-positive InfoNCE TRAINING (GRCh38 only) =====")

    total_steps = (NUM_TUPLES // BATCH_SIZE_TR) * TRAIN_EPOCHS
    step = 0

    head.train()
    for ep in range(TRAIN_EPOCHS):
        print(f"\n[Epoch {ep+1}] Generating tuples...")
        tuples = generate_multi_pos_tuples(grch_seqs, NUM_TUPLES)
        random.shuffle(tuples)

        epoch_losses = []

        for i in tqdm(range(0, NUM_TUPLES, BATCH_SIZE_TR), desc=f"Epoch {ep+1}"):
            batch = tuples[i:i+BATCH_SIZE_TR]
            if len(batch) < 2:
                continue

            anc, pos1, pos2 = zip(*batch)

            # DNABERT용 k-mer 변환
            anc_k  = [to_kmer(s, KMER) for s in anc]
            pos1_k = [to_kmer(s, KMER) for s in pos1]
            pos2_k = [to_kmer(s, KMER) for s in pos2]

            # 하나의 리스트로 합쳐서 토크나이즈
            seqs_all = anc_k + pos1_k + pos2_k   # 길이 = 3B

            batch_enc = tokenizer(
                seqs_all,
                padding=True,
                truncation=True,
                max_length=MAX_LENGTH,
                return_tensors="pt"
            ).to(device)

            optimizer.zero_grad()

            with amp.autocast(device_type="cuda", enabled=USE_FP16):
                out = backbone(**batch_enc)
                hidden_states = out.hidden_states   # list of (B', L, H)

                emb_all = head(hidden_states, batch_enc["attention_mask"])  # (3B, D)

                B = len(batch)
                e_anc  = emb_all[:B]
                e_pos1 = emb_all[B:2*B]
                e_pos2 = emb_all[2*B:3*B]

                loss = multipos_infonce_loss(e_anc, e_pos1, e_pos2, TEMPERATURE)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            epoch_losses.append(loss.item())
            step += 1

        print(f"[Epoch {ep+1}] InfoNCE loss = {np.mean(epoch_losses):.4f}")

    # -----------------------------
    # INFERENCE
    # -----------------------------
    print("\n===== Inference on test.csv (original + RC, mean) =====")
    head.eval()

    all_embs = []

    with torch.no_grad():
        for i in tqdm(range(0, len(test_seqs), BATCH_SIZE_INFER), desc="Inference"):
            batch = test_seqs[i:i+BATCH_SIZE_INFER]

            # 원본/RC 정리
            batch_clean = [clean_seq(s) for s in batch]
            batch_rc    = [reverse_complement(s) for s in batch_clean]

            batch_kmer    = [to_kmer(s, KMER) for s in batch_clean]
            batch_rc_kmer = [to_kmer(s, KMER) for s in batch_rc]

            seqs_all = batch_kmer + batch_rc_kmer

            enc = tokenizer(
                seqs_all,
                padding=True,
                truncation=True,
                max_length=MAX_LENGTH,
                return_tensors="pt"
            ).to(device)

            out = backbone(**enc)
            hidden_states = out.hidden_states

            emb_all = head(hidden_states, enc["attention_mask"])  # (2B, D)
            B = len(batch)
            emb_v1 = emb_all[:B]
            emb_v2 = emb_all[B:2*B]

            emb_mean = (emb_v1 + emb_v2) / 2.0
            all_embs.append(emb_mean.cpu().numpy())

    all_embs = np.concatenate(all_embs, axis=0)  # (N_test, D)

    # -----------------------------
    # Save submission
    # -----------------------------
    col_names = [f"emb_{i:04d}" for i in range(OUTPUT_DIM)]
    emb_df = pd.DataFrame(all_embs, columns=col_names)

    final_df = pd.concat([test_df[["ID"]], emb_df], axis=1)
    final_df.to_csv(OUT_PATH, index=False)

    print("\n✅ Saved:", OUT_PATH)


if __name__ == "__main__":
    main()

### nucleotide-transformer-v2 코드 (점수 : 0.539)

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os, random, gc
import numpy as np
import pandas as pd
from typing import List
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import amp
from transformers import AutoTokenizer, AutoModelForMaskedLM, get_cosine_schedule_with_warmup


# ======================
# CONFIG
# ======================
SEED = 42
MODEL_ID = "InstaDeepAI/nucleotide-transformer-v2-500m-multi-species"

DATA_DIR = r"C:\Users\yutt4\OneDrive\바탕 화면\인공지능\open"
EXTERNAL_CSV = os.path.join(DATA_DIR, "human_genome_train.csv")
GRCH_PATH    = os.path.join(DATA_DIR, "grch38_windows_seq.csv")
TEST_PATH    = os.path.join(DATA_DIR, "test.csv")
SAMPLE_SUB_PATH = os.path.join(DATA_DIR, "sample_submission.csv")

OUT_PATH = "submission_heavy_finetune_contrastive_max128_수정본4.csv"

# 핵심 파라미터
MAX_LENGTH = 128
OUTPUT_DIM = 512
LAST_N_LAYERS = 12       # 활용 layer 확대

TRAIN_EPOCHS = 10        # 매우 길게
NUM_TUPLES = 120000      # 튜플 크게 증가
BATCH_SIZE_TR = 16
BATCH_SIZE_INFER = 32

TEMPERATURE = 0.1

MAX_EXT_SEQ = 200_000
MAX_GRCH_SEQ = 300_000

USE_FP16 = True

# 백본 파인튜닝 깊게
UNFREEZE_BACKBONE = True
BACKBONE_LR = 1e-6
BACKBONE_WEIGHT_DECAY = 1e-6

# projection head 학습률
LR_HEAD = 5e-4
HEAD_WEIGHT_DECAY = 1e-4


# ======================
# UTILS
# ======================
def set_seed(s=42):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)


def clean(seq: str) -> str:
    seq = str(seq).upper().replace("N", "")
    return seq


def reverse_complement(seq: str) -> str:
    return seq.translate(str.maketrans("ACGT", "TGCA"))[::-1]


def apply_snv(seq: str, k: int):
    bases = ["A", "C", "G", "T"]
    arr = list(seq)
    idxs = random.sample(range(len(arr)), min(len(arr), k))
    for i in idxs:
        orig = arr[i]
        cand = [b for b in bases if b != orig]
        arr[i] = random.choice(cand)
    return "".join(arr)


def crop(seq: str):
    if len(seq) <= MAX_LENGTH:
        return seq
    s = random.randint(0, len(seq)-MAX_LENGTH)
    return seq[s:s+MAX_LENGTH]


def load_grch_stream(path, max_samples, chunksize=200000):
    seqs = []
    print("\n[GRCh38 STREAM LOAD]", path)
    for chunk in pd.read_csv(path, usecols=["seq"], chunksize=chunksize):
        tmp = [clean(s) for s in chunk["seq"].tolist()]
        tmp = [s for s in tmp if len(s) > 0]

        need = max_samples - len(seqs)
        if need <= 0:
            break
        if len(tmp) > need:
            tmp = tmp[:need]

        seqs.extend(tmp)

        del chunk
        gc.collect()

        if len(seqs) >= max_samples:
            break

    print("Loaded GRCh38:", len(seqs))
    return seqs


# ======================
# TUPLE GENERATION (강한 버전)
# ======================
def generate_contrastive_tuples(seqs: List[str], N: int):
    """
    anchor: 원본 crop
    pos   : SNV 1개
    neg   : SNV 8~12개
    → SNV 민감도 극대화 전략
    """
    rows = []
    print(f"Generating {N} contrastive tuples (strong SNV)")
    while len(rows) < N:
        a = crop(clean(random.choice(seqs)))
        if len(a) == 0:
            continue
        pos = crop(apply_snv(a, 1))
        neg = crop(apply_snv(a, random.randint(8, 12)))
        rows.append((a, pos, neg))
    return rows


# ======================
# MODEL HEAD
# ======================
def l2_norm(x):
    return x / x.norm(dim=-1, keepdim=True).clamp(min=1e-12)


class Head(nn.Module):
    def __init__(self, hid, last_n, out):
        super().__init__()
        self.last_n = last_n
        self.w = nn.Parameter(torch.zeros(last_n))

        self.proj = nn.Sequential(
            nn.Linear(hid, hid*2),
            nn.LayerNorm(hid*2),
            nn.GELU(),
            nn.Linear(hid*2, out),
        )

    def forward(self, hs, mask):
        stack = torch.stack(hs[-self.last_n:], dim=0)   # (L, B, T, H)
        w = F.softmax(self.w, dim=0).view(-1,1,1,1)
        feat = (stack * w).sum(dim=0)                   # (B, T, H)

        mask = mask.unsqueeze(-1).float()
        pooled = (feat * mask).sum(1) / mask.sum(1).clamp(min=1e-9)
        return l2_norm(self.proj(pooled))


def contrastive_loss(ea, ep, en, temp=0.1):
    B = ea.size(0)
    all_e = torch.cat([ea, ep, en], dim=0)
    sim = torch.matmul(ea, all_e.T) / temp
    labels = torch.arange(B, 2*B, device=ea.device)
    return F.cross_entropy(sim, labels)


# ======================
# MAIN
# ======================
def main():

    set_seed(SEED)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Device:", device)

    # ------------------------
    # Load external data
    # ------------------------
    df = pd.read_csv(EXTERNAL_CSV)
    ext = [clean(s) for s in df["seq"].tolist()]
    ext = [s for s in ext if len(s) > 0]
    if len(ext) > MAX_EXT_SEQ:
        idx = np.random.choice(len(ext), MAX_EXT_SEQ, replace=False)
        ext = [ext[i] for i in idx]

    # ------------------------
    # Load GRCh38 stream
    # ------------------------
    grch = load_grch_stream(GRCH_PATH, MAX_GRCH_SEQ)

    # training sequences = ext + grch
    all_seqs = ext + grch
    print("Total training seqs:", len(all_seqs))

    # ------------------------
    # Load test
    # ------------------------
    test_df = pd.read_csv(TEST_PATH)
    test_seqs = [clean(s) for s in test_df["seq"].tolist()]
    print("Test seqs:", len(test_seqs))

    # ------------------------
    # Load backbone
    # ------------------------
    tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
    gLM = AutoModelForMaskedLM.from_pretrained(MODEL_ID, trust_remote_code=True)
    gLM.to(device)

    # Freeze all first
    for p in gLM.parameters():
        p.requires_grad = False

    # 🔥 Unfreeze LAST 8 LAYERS: 24~31
    if UNFREEZE_BACKBONE:
        for name, p in gLM.named_parameters():
            if any(f"encoder.layer.{i}" in name for i in range(24, 32)):
                p.requires_grad = True

    # ------------------------
    # HEAD
    # ------------------------
    model = Head(
        hid=gLM.config.hidden_size,
        last_n=LAST_N_LAYERS,
        out=OUTPUT_DIM
    ).to(device)

    # ------------------------
    # OPTIMIZER + SCHEDULER
    # ------------------------
    params = [
        {"params": model.parameters(), "lr": LR_HEAD, "weight_decay": HEAD_WEIGHT_DECAY},
        {"params": [p for p in gLM.parameters() if p.requires_grad],
         "lr": BACKBONE_LR, "weight_decay": BACKBONE_WEIGHT_DECAY}
    ]

    optim = torch.optim.AdamW(params)
    total_steps = (NUM_TUPLES // BATCH_SIZE_TR) * TRAIN_EPOCHS
    warmup_steps = total_steps // 10  # 10% warmup

    scheduler = get_cosine_schedule_with_warmup(
        optim,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

    scaler = amp.GradScaler(enabled=USE_FP16)

    # ==========================
    # TRAIN LOOP
    # ==========================
    for ep in range(TRAIN_EPOCHS):
        print(f"\n[Epoch {ep+1}] Generating tuples...")
        tuples = generate_contrastive_tuples(all_seqs, NUM_TUPLES)
        random.shuffle(tuples)

        losses = []

        for i in tqdm(range(0, NUM_TUPLES, BATCH_SIZE_TR)):
            batch = tuples[i:i+BATCH_SIZE_TR]
            if len(batch) < 2: continue

            anc, pos, neg = zip(*batch)
            seqs = list(anc) + list(pos) + list(neg)

            enc = tok(
                seqs,
                truncation=True,
                padding=True,
                max_length=MAX_LENGTH,
                return_tensors="pt"
            ).to(device)

            optim.zero_grad()

            with amp.autocast(device_type="cuda", enabled=USE_FP16):
                out = gLM(**enc, output_hidden_states=True)
                emb = model(out.hidden_states, enc["attention_mask"])

                B = len(batch)
                ea = emb[:B]
                ep = emb[B:2*B]
                en = emb[2*B:3*B]

                loss = contrastive_loss(ea, ep, en, TEMPERATURE)

            scaler.scale(loss).backward()
            scaler.step(optim)
            scaler.update()
            scheduler.step()

            losses.append(loss.item())

        print(f"[Epoch {ep+1}] loss:", np.mean(losses))

    # ==========================
    # INFERENCE
    # ==========================
    print("\n[Inference]")

    final_embs = []
    for i in tqdm(range(0, len(test_seqs), BATCH_SIZE_INFER)):
        batch = test_seqs[i:i+BATCH_SIZE_INFER]

        enc = tok(
            batch,
            truncation=True,
            padding=True,
            max_length=MAX_LENGTH,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            out = gLM(**enc, output_hidden_states=True)
            emb = model(out.hidden_states, enc["attention_mask"])
            final_embs.append(emb.cpu().numpy())

    final_embs = np.concatenate(final_embs, axis=0)
    cols = [f"emb_{i:04d}" for i in range(OUTPUT_DIM)]
    df_emb = pd.DataFrame(final_embs, columns=cols)

    out = pd.concat([test_df[["ID"]], df_emb], axis=1)
    out.to_csv(OUT_PATH, index=False)

    print("\nSaved:", OUT_PATH)


if __name__ == "__main__":
    main()